# Mass Shooting Analysis 4: Aggregating MSI data by State and Wrangling Socio-Economic and Gun Use Data

In [1]:
import json
import pandas as pd
import numpy as np
import random
from pathlib import Path
import seaborn as sns
from matplotlib_inline.backend_inline import set_matplotlib_formats
import myst_nb
import wikipedia as wp
import math

## Data Grouped by State and Aggregated

In [2]:
mst_df = pd.read_csv('MSA Data\mst14-24_data_big_inc_rem.csv',index_col=0, encoding='utf-8') # Load pre-prepped data
scope = 'USA, 2014-24'

In [3]:
gby_state = mst_df.groupby(['state']).agg(
    no_ms_inc = pd.NamedAgg(column="killed", aggfunc="count"),
    total_inj = pd.NamedAgg(column="wounded", aggfunc="sum"),
    total_kld = pd.NamedAgg(column="killed", aggfunc="sum"), 
    total_inj_kill = pd.NamedAgg(column="total", aggfunc="sum")
).reset_index()
gby_state.head(5)

,state,no_ms_inc,total_inj,total_kld,total_inj_kill
0,AK,9,20,20,40
1,AL,176,713,210,923
2,AR,70,307,85,392
3,AZ,78,260,132,392
4,CA,524,1949,678,2627


In [4]:
gby_state['kpi'] = gby_state['total_kld'] / gby_state['no_ms_inc']
gby_state['wpi'] = gby_state['total_inj'] / gby_state['no_ms_inc']

In [5]:
top10_states = gby_state.nlargest(10, 'no_ms_inc').reset_index(drop=True)

In [6]:
top10_states

,state,no_ms_inc,total_inj,total_kld,total_inj_kill,kpi,wpi
0,IL,553,2271,455,2726,0.822785,4.106691
1,CA,524,1949,678,2627,1.293893,3.719466
2,TX,481,1774,780,2554,1.621622,3.688150
3,FL,358,1391,450,1841,1.256983,3.885475
4,PA,285,1115,267,1382,0.936842,3.912281
5,LA,264,1097,239,1336,0.905303,4.155303
6,GA,258,947,300,1247,1.162791,3.670543
7,NY,248,1018,167,1185,0.673387,4.104839
8,OH,248,976,296,1272,1.193548,3.935484
9,NC,211,716,269,985,1.274882,3.393365


In [7]:
big_ms_states = gby_state[gby_state['no_ms_inc'] > 99] # Dropping States with lower than 100 incidents due to laws of small numbers in wpi/kpi stats
len(big_ms_states)

20

In [8]:
big_ms_states.nlargest(5, 'kpi').reset_index(drop=True)

,state,no_ms_inc,total_inj,total_kld,total_inj_kill,kpi,wpi
0,TX,481,1774,780,2554,1.621622,3.688150
1,CA,524,1949,678,2627,1.293893,3.719466
2,NC,211,716,269,985,1.274882,3.393365
3,FL,358,1391,450,1841,1.256983,3.885475
4,VA,137,503,168,671,1.226277,3.671533


# Gun Ownership & Gun Violence Stats from the States

In [9]:
html = wp.page("Gun death and violence in the United States by state").html().encode("UTF-8")
gdv1_df = pd.read_html(html)[1]
gdv1_df.head(5)

,Location,Gun suicide rate,Suicide rate,Gun homicide rate,Homicide rate,% gun at home
0,United States,7.9,14.5,6.3,7.8,[a]
1,Wyoming,23.7,32.8,1.7,2.8,61%
2,Montana,21.6,31.7,2.7,4.2,65%
3,Alaska,19.4,30.0,4.2,6.7,57%
4,New Mexico,14.4,25.2,10.9,14.5,36%


In [10]:
html = wp.page("Gun death and violence in the United States by state").html().encode("UTF-8")
gdv2_df = pd.read_html(html)[2]
gdv2_df.head(5)

,State,Gun deaths,Suicide,Homicide,Accident,Law
0,United States,48830,26328,20958.0,549.0,537.0
1,Texas,4613,2528,1942.0,53.0,38.0
2,California,3576,1575,1861.0,32.0,89.0
3,Florida,3142,1928,1150.0,18.0,25.0
4,Georgia,2200,1115,1021.0,25.0,22.0


In [11]:
gdv2_df['Law'].isna().value_counts(), gdv2_df['Accident'].isna().value_counts()

(Law
 True     32
 False    20
 Name: count, dtype: int64,
 Accident
 False    29
 True     23
 Name: count, dtype: int64)

In [12]:
gdv2_df['Gun deaths'] = gdv2_df['Gun deaths'].astype('Int64')
gdv2_df['Suicide'] = gdv2_df['Suicide'].fillna(0).astype('Int64')
gdv2_df['Homicide'] = gdv2_df['Homicide'].fillna(0).astype('Int64')

In [13]:
gdv_df = gdv1_df.merge(gdv2_df, 
                 left_on='Location',
                 right_on='State',
                 how='left')       # left join instead of inner to keep any territory w/o data
gdv_df.head(4)

,Location,Gun suicide rate,Suicide rate,Gun homicide rate,Homicide rate,% gun at home,State,Gun deaths,Suicide,Homicide,Accident,Law
0,United States,7.9,14.5,6.3,7.8,[a],United States,48830,26328,20958,549.0,537.0
1,Wyoming,23.7,32.8,1.7,2.8,61%,Wyoming,155,137,10,NaN,NaN
2,Montana,21.6,31.7,2.7,4.2,65%,Montana,280,239,30,NaN,NaN
3,Alaska,19.4,30.0,4.2,6.7,57%,Alaska,182,142,31,NaN,NaN


In [14]:
gdv_df = gdv_df[['Location', 
                 'Gun suicide rate', 
                 'Suicide rate', 
                 'Gun homicide rate',	
                 'Homicide rate',	
                 '% gun at home',	
                 'Gun deaths',
                 'Suicide',
                 'Homicide']]

In [15]:
gdv_df.dtypes

Location              object
Gun suicide rate     float64
Suicide rate         float64
Gun homicide rate     object
Homicide rate        float64
% gun at home         object
Gun deaths             Int64
Suicide                Int64
Homicide               Int64
dtype: object

In [16]:
gdv_df['Gun homicide rate'] = pd.to_numeric(gdv_df['Gun homicide rate'], errors='coerce')

In [17]:
gdv_df['% gun at home'] = gdv_df['% gun at home'].str.rstrip('%')

In [18]:
gdv_df['gun_at_home'] = pd.to_numeric(gdv_df['% gun at home'], errors='coerce')

In [19]:
gdv_df['gun_at_home'] = gdv_df['gun_at_home'] / 100

In [20]:
gdv_df.head(5)

,Location,Gun suicide rate,Suicide rate,Gun homicide rate,Homicide rate,% gun at home,Gun deaths,Suicide,Homicide,gun_at_home
0,United States,7.9,14.5,6.3,7.8,[a],48830,26328,20958,NaN
1,Wyoming,23.7,32.8,1.7,2.8,61,155,137,10,0.61
2,Montana,21.6,31.7,2.7,4.2,65,280,239,30,0.65
3,Alaska,19.4,30.0,4.2,6.7,57,182,142,31,0.57
4,New Mexico,14.4,25.2,10.9,14.5,36,578,305,230,0.36


In [21]:
gdv_df.dtypes

Location              object
Gun suicide rate     float64
Suicide rate         float64
Gun homicide rate    float64
Homicide rate        float64
% gun at home         object
Gun deaths             Int64
Suicide                Int64
Homicide               Int64
gun_at_home          float64
dtype: object

In [22]:
gdv_df.to_csv('gdv_df.csv') # save a copy as wikipedia table numbers had changed so data can be loaded instead of retrieved

In [23]:
states_df = pd.read_csv('MSA Data\States_abbr_pop_size.csv', encoding='utf-8')

In [24]:
states_df.head(5)

,state_us,state_abb,population,area_sqm,area_sqkm,reps
0,Alabama,AL,"5,024,279","52,420","135,767",7
1,Alaska,AK,"733,391","665,384","1,723,337",1
2,Arizona,AZ,"7,151,502","113,990","295,234",9
3,Arkansas,AR,"3,011,524","53,179","137,732",4
4,California,CA,"39,538,223","163,695","423,967",52


In [25]:
import locale
locale.setlocale(locale.LC_NUMERIC, 'en_GB')
states_df["population"] = states_df["population"].apply(locale.atoi)
states_df["area_sqkm"] = states_df["area_sqkm"].apply(locale.atoi)

In [26]:
ms_sts = gby_state.merge(states_df, 
                 left_on='state',
                 right_on='state_abb',
                 how='left')       # left join instead of inner to keep any territory w/o data
ms_sts.head(4)

,state,no_ms_inc,total_inj,total_kld,total_inj_kill,kpi,wpi,state_us,state_abb,population,area_sqm,area_sqkm,reps
0,AK,9,20,20,40,2.222222,2.222222,Alaska,AK,733391,"665,384",1723337,1
1,AL,176,713,210,923,1.193182,4.051136,Alabama,AL,5024279,"52,420",135767,7
2,AR,70,307,85,392,1.214286,4.385714,Arkansas,AR,3011524,"53,179",137732,4
3,AZ,78,260,132,392,1.692308,3.333333,Arizona,AZ,7151502,"113,990",295234,9


In [27]:
gdv_df.head(3)

,Location,Gun suicide rate,Suicide rate,Gun homicide rate,Homicide rate,% gun at home,Gun deaths,Suicide,Homicide,gun_at_home
0,United States,7.9,14.5,6.3,7.8,[a],48830,26328,20958,NaN
1,Wyoming,23.7,32.8,1.7,2.8,61,155,137,10,0.61
2,Montana,21.6,31.7,2.7,4.2,65,280,239,30,0.65


In [28]:
ms_sts = ms_sts.merge(gdv_df, 
                 left_on='state_us',
                 right_on='Location',
                 how='left')      # left join instead of inner to keep any territory w/o data
ms_sts.head(5)

,state,no_ms_inc,total_inj,total_kld,total_inj_kill,kpi,wpi,state_us,state_abb,population,...,Location,Gun suicide rate,Suicide rate,Gun homicide rate,Homicide rate,% gun at home,Gun deaths,Suicide,Homicide,gun_at_home
0,AK,9,20,20,40,2.222222,2.222222,Alaska,AK,733391,...,Alaska,19.4,30.0,4.2,6.7,57,182,142,31,0.57
1,AL,176,713,210,923,1.193182,4.051136,Alabama,AL,5024279,...,Alabama,12.4,16.4,12.9,14.8,53,1315,623,650,0.53
2,AR,70,307,85,392,1.214286,4.385714,Arkansas,AR,3011524,...,Arkansas,12.9,20.4,9.3,11.1,52,698,391,281,0.52
3,AZ,78,260,132,392,1.692308,3.333333,Arizona,AZ,7151502,...,Arizona,12.1,20.3,5.9,7.7,36,1365,879,430,0.36
4,CA,524,1949,678,2627,1.293893,3.719466,California,CA,39538223,...,California,4.0,10.6,4.7,6.4,16,3576,1575,1861,0.16


In [29]:
ms_sts.dtypes

state                 object
no_ms_inc              int64
total_inj              int64
total_kld              int64
total_inj_kill         int64
kpi                  float64
wpi                  float64
state_us              object
state_abb             object
population             int64
area_sqm              object
area_sqkm              int64
reps                   int64
Location              object
Gun suicide rate     float64
Suicide rate         float64
Gun homicide rate    float64
Homicide rate        float64
% gun at home         object
Gun deaths             Int64
Suicide                Int64
Homicide               Int64
gun_at_home          float64
dtype: object

In [30]:
ms_sts['ms_rate'] = ms_sts['no_ms_inc'] / ms_sts['population'] * 100000

In [31]:
ms_sts['pop_density'] = ms_sts['population'] / ms_sts['area_sqkm']

In [32]:
ms_sts['gun_death_rate'] = ms_sts['Gun deaths'] / ms_sts['population'] * 100000

In [33]:
ms_sts['kld_to_wd'] = ms_sts['total_kld'] / ms_sts['total_inj_kill']

In [34]:
ms_sts['prop_suic_w_gun'] = ms_sts['Gun suicide rate'] / ms_sts['Suicide rate']

In [35]:
ms_sts['prop_hom_w_gun'] = ms_sts['Gun homicide rate'] / ms_sts['Homicide rate']

In [36]:
ms1_sts = ms_sts[['state_us', 'population', 'area_sqkm', 'pop_density',
                'ms_rate', 'no_ms_inc', 'total_inj', 'total_kld', 'total_inj_kill', 'wpi', 'kpi', 'kld_to_wd',
                'Suicide rate', 'Gun suicide rate',  'Suicide', 'prop_suic_w_gun',
                'Homicide rate', 'Gun homicide rate', 'Homicide', 'prop_hom_w_gun',
                'Gun deaths', 'gun_death_rate', 'gun_at_home']]

In [37]:
ms_sts.columns

Index(['state', 'no_ms_inc', 'total_inj', 'total_kld', 'total_inj_kill', 'kpi',
       'wpi', 'state_us', 'state_abb', 'population', 'area_sqm', 'area_sqkm',
       'reps', 'Location', 'Gun suicide rate', 'Suicide rate',
       'Gun homicide rate', 'Homicide rate', '% gun at home', 'Gun deaths',
       'Suicide', 'Homicide', 'gun_at_home', 'ms_rate', 'pop_density',
       'gun_death_rate', 'kld_to_wd', 'prop_suic_w_gun', 'prop_hom_w_gun'],
      dtype='object')

In [38]:
gun_sts = ms_sts[['state', 'no_ms_inc',
                'Homicide rate', 'Gun homicide rate', 'Homicide', 'prop_hom_w_gun',
                'Suicide rate', 'Gun suicide rate',  'Suicide', 'prop_suic_w_gun',
                'Gun deaths', 'gun_death_rate', 'gun_at_home']]
gun_sts.head(2)

,state,no_ms_inc,Homicide rate,Gun homicide rate,Homicide,prop_hom_w_gun,Suicide rate,Gun suicide rate,Suicide,prop_suic_w_gun,Gun deaths,gun_death_rate,gun_at_home
0,AK,9,6.7,4.2,31,0.626866,30.0,19.4,142,0.646667,182,24.81623,0.57
1,AL,176,14.8,12.9,650,0.871622,16.4,12.4,623,0.756098,1315,26.17291,0.53


In [40]:
# gun_sts.to_csv('MSA Data\\us_gun_data_by_state.csv', encoding='utf-8')

In [46]:
ms1_sts.dtypes

state_us              object
population             int64
area_sqkm              int64
pop_density          float64
ms_rate              float64
no_ms_inc              int64
total_inj              int64
total_kld              int64
total_inj_kill         int64
wpi                  float64
kpi                  float64
kld_to_wd            float64
Suicide rate         float64
Gun suicide rate     float64
Suicide                Int64
prop_suic_w_gun      float64
Homicide rate        float64
Gun homicide rate    float64
Homicide               Int64
prop_hom_w_gun       float64
Gun deaths             Int64
gun_death_rate       Float64
gun_at_home          float64
dtype: object

In [47]:
labels = {'population':'Population (State)',
          'area_sqkm':'Area of the State in Square Kilometers',
          'pop_density':'Population Density (/km^2)',
          'ms_rate': 'Mass Shooting Incidents per 100k (2014-24)',
          'no_ms_inc': 'Number of Mass Shooting Incidents',
          'total_inj':'Total of People Injured in Mass Shooting Incidents',
          'total_kld':'Total of People Killed in Mass Shooting Incidents',
          'total_inj_kill':'Total People Injured or Killed in Mass Shooting Incidents',
          'kpi':'Avg # of People Killed per Mass Shooting Incident',
          'wpi':'Avg # of People Injured per Mass Shooting Incident',
          'kld_to_wd':'Proportion of Victims Killed in a MS Incident',
          'Suicide rate':'Rate of Suicide per 100k People',
          'Gun suicide rate':'Rate of Suicide with a Gun per 100k People',
          'prop_suic_w_gun':'Proportion of Suicides Commited with a Gun, 2021 (CDC data)',
          'Homicide rate':'Rate of Homicide per 100k People',
          'Homicide':'Total Number of Homicide (Victims)',
          'Gun homicide rate':'Rate of Homicide with a Gun per 100k People',
          'prop_hom_w_gun':'Proportion of Homicides Commited with a Gun, 2021 (CDC data)',
          'gun_death_rate':'Rate of Gunshot Deaths per 100k People',
          'Gun deaths':'Total Number of Gun Deaths (Victims)',
          'gun_at_home':'Proportion of Households in the State that own a Gun'
          }

In [48]:
ms1_sts.head(5)

,state_us,population,area_sqkm,pop_density,ms_rate,no_ms_inc,total_inj,total_kld,total_inj_kill,wpi,...,Gun suicide rate,Suicide,prop_suic_w_gun,Homicide rate,Gun homicide rate,Homicide,prop_hom_w_gun,Gun deaths,gun_death_rate,gun_at_home
0,Alaska,733391,1723337,0.425564,1.227176,9,20,20,40,2.222222,...,19.4,142,0.646667,6.7,4.2,31,0.626866,182,24.81623,0.57
1,Alabama,5024279,135767,37.006629,3.502990,176,713,210,923,4.051136,...,12.4,623,0.756098,14.8,12.9,650,0.871622,1315,26.17291,0.53
2,Arkansas,3011524,137732,21.865100,2.324405,70,307,85,392,4.385714,...,12.9,391,0.632353,11.1,9.3,281,0.837838,698,23.177634,0.52
3,Arizona,7151502,295234,24.223165,1.090680,78,260,132,392,3.333333,...,12.1,879,0.596059,7.7,5.9,430,0.766234,1365,19.0869,0.36
4,California,39538223,423967,93.257784,1.325300,524,1949,678,2627,3.719466,...,4.0,1575,0.377358,6.4,4.7,1861,0.734375,3576,9.044413,0.16


In [49]:
ms1_sts = ms1_sts.drop(ms1_sts[ms1_sts['state_us'] == 'Puerto Rico'].index)

In [50]:
ms1_sts = ms1_sts.drop(ms1_sts[ms1_sts['state_us'] == 'Virgin Islands'].index)

In [51]:
ms1_sts = ms1_sts.drop(index=7)        # Remove District of Columbia - extreme outlier

In [52]:
ms1_sts.head(3)

,state_us,population,area_sqkm,pop_density,ms_rate,no_ms_inc,total_inj,total_kld,total_inj_kill,wpi,...,Gun suicide rate,Suicide,prop_suic_w_gun,Homicide rate,Gun homicide rate,Homicide,prop_hom_w_gun,Gun deaths,gun_death_rate,gun_at_home
0,Alaska,733391,1723337,0.425564,1.227176,9,20,20,40,2.222222,...,19.4,142,0.646667,6.7,4.2,31,0.626866,182,24.81623,0.57
1,Alabama,5024279,135767,37.006629,3.502990,176,713,210,923,4.051136,...,12.4,623,0.756098,14.8,12.9,650,0.871622,1315,26.17291,0.53
2,Arkansas,3011524,137732,21.865100,2.324405,70,307,85,392,4.385714,...,12.9,391,0.632353,11.1,9.3,281,0.837838,698,23.177634,0.52


In [55]:
ms1_sts.to_csv('MSA Data\\us_states_demo_gun_vio_df.csv', encoding='utf-8')